# Michaelis-Menten Model Calibration Notebook

Based on PTemPest example written in Matlab [here](https://github.com/RuleWorld/ptempest/tree/master/examples/michment)

## Overview

This system describes the 1) reversible binding of an enzyme to substrate and 2) production of substrate product which is defined in the following scheme:  
$$E + S \rightleftharpoons^{k_f}_{k_r} ES \longrightarrow^{k_{cat}} E + P$$  

Assuming total enzyme concentration is significantly smaller than substrate concentration (i.e., $[E]_T \ll [S]$), the rate is defined as:  
$$\frac{d[P]}{dt} = \frac{k_{cat}[E]_T[S]}{K_M + [S]}$$
where $K_M = \frac{k_{cat} + k_r}{k_f}$

## Model Calibration

The following notebook calibrates the Michaelis-Menten model system using synthetically generated data with 1% Gaussian error. Here, we test the following inference methods:
1. Metropolis-Hastings (`pyPESTO`)
2. Parallel-Tempering MCMC (`pyPESTO`)
3. Nested Sampling (`dynesty`)
4. Sequential Monte Carlo (`pocoMC`)
5. Preconditioned Monte Carlo (`pocoMC`)

### Load relevant packages

In [ ]:
import os
import time
import pickle
import roadrunner
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import qmc
from multiprocessing import Pool

import pypesto
import pocomc as pc
import dynesty as dy
import pypesto.engine as eng
import pypesto.sample as sample
import pypesto.store as store
import pypesto.optimize as optimize
from pypesto.ensemble import Ensemble

### Defining the `Model` class

The model class has the following attributes:
1. `x_n` : `int`
    Number of species in the model
2. `fit_x0` : `bool`
    Whether the initial conditions are to be estimated and are therefore include in `theta` args.
3. `x0` : `list[float], optional`
    The initial conditions of model species
4. `theta_n` : `int`
    Number of parameters to be fit. This includes ALL parameters to be estimated which MAY include initial conditions and the standard deviation of the model species. The order of the model parameters in this list is assumed to be as follows:
    1. ODE equation parameters
    2. Initial conditions (denoted $x_\#$, **optional**)
    3. Species standard deviations (denoted $\sigma_\#$)
5. `theta_true` : `list[float]` of shape `(theta_n)`
    True theta values of the model system. The order of 
6. `theta_names` : `list[str]` of shape `(theta_n)`
    Name of parameters for plotting purposes
7. `lower_bnds` : `list[float]` of shape `(theta_n)`
    Lower bounds of parameter values 
8. `upper_bnds` : `list[float]` of shape `(theta_n)`
    Upper bounds of parameter values 
9. `ts` : `list[float]` 
    Experimental data times 
10. `data` : `list[float]` of shape `(ts, x_n)` 
    Experimental data used for model calibration
11. `librr_model` : TBD 

In [ ]:
class Model:
    def __init__(self, opts): #initial settings
        for key in opts: #loops for all labels in the list 'key'
            setattr(self, key, opts[key]) #creates a dictionary where 'key' are the list of labels & 'ops[key]' are the values

    def __call__(self, theta_new):
        theta_new = theta_new
        res = self.log_likelihood(theta_new)
        return res
    
    def change_and_run(self, model_param, x0):
        rr = self.librr_model
        rr.resetAll()
        rr.integrator.absolute_tolerance = 5e-10
        rr.integrator.relative_tolerance = 1e-8

        for spec_name, val in zip(self.librr_species, x0):
            init_species_string = f"init([{spec_name}])"
            rr[init_species_string] = val
            rr.reset()
        
        for name, value in zip(self.librr_theta, model_param):
            rr[name] = value
            rr.reset() # forces initial conditions and BNGL functions to be re-evaluated
        
        t_span = (self.ts[0], self.ts[-1])
        trajs = rr.simulate(t_span[0], t_span[1], int(t_span[1]*100+1))
        return trajs    
        
    def call_sim(self, model_param = None, x0 = None, return_all_species=False): #takes in candidate parameters then solves the ode
        if model_param is None:
            model_param= self.theta_true[:self.ODE_params_n]  
        if x0 is None:
            x0 = self.x0 
        
        trajs = self.change_and_run(model_param, x0)
        if return_all_species:
            return trajs
        
        sim_ts = trajs[:, 0]
        species = trajs[:, 1:]
        # ! TO DO: Assumes simulation includes data ts which is not always the case
        # ! Need to add interpolation for times not included in simulation
        t_idxs = np.where(np.in1d(sim_ts, self.ts))[0]
        return_species = species[t_idxs, self.observable_idxs]
        return return_species
    
    def log_prior(self, theta_new): 
        bools = [(low <= i <= high) for i,low,high in zip(theta_new, self.lower_bnds, self.upper_bnds)] #if generated values are within bounds
        all_in_range = np.all(bools) #if all values are true, then output is true
        if all_in_range: 
            return 0.0 
        return -np.inf #if even one parameter out of bounds, it's false, and returns -infinity

    def log_likelihood(self, theta_new): #how good is this canidate parameter fitting my data (maximize it)
        model_param = theta_new[:self.ODE_params_n] 
        if self.fit_x0: 
            x0 = theta_new[self.ODE_params_n:(self.ODE_params_n + self.x_n)] #sets x0 to 'theta_true' x0 values
        else:
            x0 = self.x0

        if self.fit_sigma:
            sigma = theta_new[-len(self.observable_idxs):] #observable index related to sigma
        else:
            sigma = [1] * len(self.observable_idxs) #makes all sigmas default to 1
        y = self.call_sim(model_param=model_param, x0=x0) #sets y to the y results of solving ODE
        data = self.data #sets data

        # Calculate likelihood
        term1 = -0.5 * np.log(2*np.pi*np.square(sigma))
        term2 = np.square(np.subtract(y, data)) / (2*np.square(sigma))
        logLH = np.sum(term1 - term2)
        return logLH

### Defining the dictionary used to create the Michaelis-Menten problem in the `Model` class

In [ ]:
# Michaelis-Menten Model Options
mod_opts = {} #creates a dictionary
mod_opts['theta_n'] = 3 #total number of values in big array
mod_opts['ODE_params_n'] = 3 # num ODE params
mod_opts['x_n'] = 4 # num species
mod_opts['sigma_n'] = 0 # num sigmas
mod_opts['theta_labels'] = ['$k_1$', '$k_2$', '$k_3$'] # parameter labels

# ! TO DO: we currently assume it fit_x0, then ALL species initial conditions are fit
mod_opts['fit_x0'] = False # fit initial conditions?
mod_opts['x0'] = [600,6,0,0] # initial conditions (if given)
mod_opts['observable_idxs'] = [3] # indices of outputs containing the observables
mod_opts['fit_sigma'] = False #fit sigma?
mod_opts['theta_true'] = [-2.77, -1.0, -2.0] #guess param values(in log)
mod_opts['lower_bnds'] = [-3,-1,-3] #lower bounds(in log)
mod_opts['upper_bnds'] = [1,3,3] #upper bounds(in log)

# load data for model
mod_df = pd.read_csv('mm_data.csv', header=0, delimiter=",") #reads in data file
mod_opts['ts'] = mod_df['t'].values #sets data under 't' in csv as 'ts'
data = mod_df['P'].values
mod_opts['data'] = data
# Load in SBML model using libroadrunner
sbml_file = "mm_sbml.xml"
librr_model = roadrunner.RoadRunner(sbml_file)
librr_theta = ["log_k1", "log_k2", "log_k3"]
librr_species = ["S1", "S2", "S3", "S4"]
librr_labels = ["Substrate", "Enzyme", "Substrate-Enzyme", "Product"]

mod_opts['librr_model'] = librr_model
mod_opts['librr_theta'] = librr_theta
mod_opts["librr_species"] = librr_species

mod = Model(mod_opts)

In [ ]:
true_trajs = mod.call_sim(return_all_species=True)
bad_trajs = mod.call_sim(x0=[200,6,0,300], model_param=[0.0,2.0,-2.0], return_all_species=True)

plt.figure()
plt.plot(true_trajs[:, 0], true_trajs[:, 4], label="True Solution")
plt.plot(bad_trajs[:, 0], bad_trajs[:, 4], label="Poor Fit")
plt.plot(mod.ts, mod.data, 'go', label="Synthetic Data")
plt.xlabel("Time");
plt.ylabel("[P]");
plt.legend(); 

In [ ]:
# Sanity checks with log_prior
test_theta = [-3,3,3] # This should return 0.0 
print(mod.log_prior(test_theta))
test_bad_theta1 = [-4, 1, 1] # should return -np.inf
print(mod.log_prior(test_bad_theta1))
test_bad_theta2 = [1, 4, 3] # should return -np.inf
print(mod.log_prior(test_bad_theta2))
test_bad_theta3 = [0, 0, 3.1] # should return -np.inf
print(mod.log_prior(test_bad_theta3))
print("--------------------")
# Sanity checks with log_likelihood
print(mod.log_likelihood(mod.theta_true))
print(mod.log_likelihood([-20,2,-5])) 